In [ ]:
# Assuming X, y are numpy arrays
def cross_validate_model(n_dim, settings, n_splits=5, load_saved_result=False, save_model=True, **kwargs):
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=model_settings['random_state'])

    performances = []

    for n_dim in n_dim:
        # Code to generate data for different dimensional synthetic datasets
        pattern = kwargs.get('pattern', 'moon')  # default pattern
        n_samples = kwargs.get('n_samples', 1500)
        noise = kwargs.get('noise', 0.1)
        increase_method = kwargs.get('increase_method', 'linear')

        X, y, orig_data = generate_data(pattern, n_samples, noise, n_dim, increase_method, random_state=model_settings['random_state'])
        # One-hot encode the labels
        y_one_hot = np.zeros((y.shape[0], len(np.unique(y))))
        y_one_hot[np.arange(y.shape[0]), np.uint(y)] = 1

        # Convert to PyTorch tensors
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y_one_hot, dtype=torch.float32)
        labels = torch.tensor(y, dtype=torch.float32)

        p, r, f = [], [], []
        for train_idx, test_idx in kfold.split(X, y):
            # "X[train_ix]" will have to be modified to extract the relevant synthetic rows
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            labels_train, labels_test = labels[train_idx], labels[test_idx]

            # Here you convert to PyTorch tensors, etc., and pass into your network
            # metrics = train_and_evaluate_model() assuming it returns a dict or other
            # performance data. Replace with the appropiate flow of your process.
            settings['data_dim'] = X_train.shape[-1]
            batch_shape = torch.Size([settings['data_dim']])
            if load_saved_result is False:
                model = create_LDGD_model(X_train, y_train, settings, x_init='pca')

                losses, history_train = model.train_model(yn=X_train, ys=y_train,
                                                          epochs=settings['num_epochs_train'],
                                                          batch_size=settings['batch_size'])
                if save_model is True:
                    model.save_wights(path_save='./saved_models/', file_name=f"model_synthetic_{num_dimension}")
                predictions, metrics, history_test = model.evaluate(yn_test=X_test, ys_test=labels_test,
                                                                epochs=settings['num_epochs_test'])

                winsound.Beep(freq, duration)
            else:
                model = create_LDGD_model(X_train, y_train, settings)
                model.load_weights(path_save='./saved_models/', file_name=f'model_synthetic_{num_dimension}.pth')
                predictions, metrics, history_test = model.evaluate(yn_test=X_test, ys_test=labels_test,
                                                                epochs=settings['num_epochs_test'])

            # Compute or accumulate measures (precision, recall, F1 score)
            p.append(metrics['precision'])
            r.append(metrics['recall'])
            f.append(metrics['f1_score'])

        # Form the list of dataframe rows for the round of n_dim rounds
        measures = (np.mean(p), np.std(p), np.mean(r), np.std(r), np.mean(f), np.std(f))
        round_result = {"dataset": f"synthetic {n_dim}", "precision": f"{measures[0]:.2f} ± {measures[1]:.2f}",
                        "recall": f"{measures[2]:.2f} ± {measures[3]:.2f}",
                        "f score": f"{measures[4]:.2f} ± {measures[5]:.2f}"}
        performances.append(round_result)

    # Creating DataFrame from performance data
    df_performances = pd.DataFrame(performances)
    return df_performances